In [ ]:
from os import system
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False    
if IN_COLAB:
    print('NOTEBOOK RUNNING ON COLAB')
    system('wget https://github.com/patrickhaddadteaching/TRNG_ex2/raw/main/exo2_src.py')
    system('wget https://github.com/patrickhaddadteaching/TRNG_ex2/raw/main/mo_trng.PNG')

### Multiple oscillators-based TRNGs

The Multiple oscillators-based TRNG (MO-TRNG) is a possible way to increase elementary oscillator-based TRNG (EO-TRNG) entropy. It consist in using a larger number of sources of randomness, i.e. number of oscillators.
Fig. 1 shows the resulting multi-oscillator based TRNG (MO-TRNG), in which outputs of  $m$ oscillators are sampled by $m$ flip-flops and XOR-ed to obtain one output bit. This kind of generator can be considered as an ensemble of $m$ EO-TRNGs.
</p>
<p align="center" width="100%">
    <img width="50%" src="./mo_trng.PNG"> 
</p>            
<p style="text-align: center;"><b>Figure 1: Multi-oscillator-based TRNG (MO-TRNG)</b></p>

<p>
Rather than it is more complex than the EO-TRNG, the designer has very limited opportunities to to obtain a high entropy rate:
    <ol>
        <li> The designer can still do so by increasing the value of the division factor $D$ at the cost of the reduced output bit rate. </li>         
        <li> The designer can alternatively increase the the number $m$ of xored oscillators </li> 
    </ol>
</p>        
<p>    
    
### The exercice
<p> In this exercise, you can analyze the impact of input parameters of the MO-TRNG on blackbox statistical tests. 
You can execute the last cell of this notebook and observe how the number of xored oscillators, the jitter variance, and the accumulation time affect the results of the 5 black box tests included in the AIS31 test suite (four of which are also included in the FIPS 140-1).</p>

<p> By default $T_{0}=1.875$ ns and the defaults $\{T_{j}\}_{j=1 \cdots 8}$ value are sumurized in the below table.</p>

<div class="row">
  <div class="column"></div>
  <div class="column"></div>
</div>

<table>
  <tr>
    <td>T<sub>1</sub></td>
    <td>3.125 ns</td>
  </tr>
    
  <tr>
    <td>T<sub>2</sub></td>
    <td>4.375 ns</td>
  </tr>
 
  <tr>
    <td>T<sub>3</sub></td>
    <td>6.875 ns</td>
  </tr>
    
  <tr>
    <td>T<sub>4</sub></td>
    <td>8.125 ns</td>
  </tr>
 
  <tr>
    <td>T<sub>5</sub></td>
    <td>10.625 ns</td>
  </tr>
    
  <tr>
    <td>T<sub>6</sub></td>
    <td>11.875 ns</td>
  </tr>
 
  <tr>
    <td>T<sub>7</sub></td>
    <td>14.375 ns</td>
  </tr>
    
  <tr>
    <td>T<sub>8</sub></td>
    <td>18.125 ns</td>
  </tr>
 
</table> 
<p>$T_{0}$ and  $\{T_{j}\}_{j=1 \cdots 8}$ can be changed by setting them to the desired values in the next cell of this notebook</p> </p>

### Your job
<p>The Graal of a TRNG designer is to find the smallest TRNG (i.e. lowest $m$), generating random numbers with the highest throuput (i.e. lowest $D$).
For this reason, your goal is to find the smallest $D$ for which the MO-TRNG would successfully pass all black box tests.</p>

#### your 1<sup>st</sup> mission
<ul>
  <li>Let fix $\frac{\sigma_{tot}}{T_{1}}$ to 0.001 wich is the most relevant jitter value observed with inverters based oscilators implemented in FPGA's [Ref to be completed]</li>
</ul>
<ul>
<li style="color:green">Your goal for this mission is to determine the smallest $D$ for which the MO-TRNG would successfully pass all black box tests.
    <ul>
    <li> for $m=1$</li>
    <li> for $m=2$</li>
    <li> for $m=3$</li>
    <li> for $m=4$</li>
    <li> for $m=5$</li>
    <li> for $m=6$</li>
    <li> for $m=7$</li>
    <li> for $m=8$</li>
    </ul>    
    </li>
</ul>


#### your 2<sup>nd</sup> mission
<ul>
<li><p style="color:green">Reproduce the 1<sup>st</sup> mission for $\frac{\sigma_{tot}}{T_{1}}=0.01$ </p>.</li>
</ul>

#### your 3<sup>rd</sup> mission
<ul>
<li><p style="color:green">Reproduce the 1<sup>st</sup> mission for $\frac{\sigma_{tot}}{T_{1}}=0$ </p>.</li>
</ul>


### Conclusion
<ul>
    <li><p style="color:red"> How the jitter variance and the number of xored oscillators impact the throughput of a MO-TRNG successfully passing the 5 black box tests included in the AIS31 test suite (four of which are also included in the FIPS 140-1) ?</p></li>
    <li><p style="color:red"> What can be drawn from the black box tests success when $\frac{\sigma_{tot}}{T_{1}}=0$ ?</p></li>
    
</ul>


In [ ]:
#T0=
#v_T=[]
from exo5_src import *
display(v_box_top)